In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Define o diretório em que o csv será salvo
PATH_RESOURCES = os.path.join(os.path.dirname(os.getcwd()), "resources/data/")

if PATH_RESOURCES not in os.getcwd():
    os.makedirs(PATH_RESOURCES, exist_ok=True)

In [3]:
# Definição do número de registros
n = 50000  

In [4]:
# Gerar datas entre 01/01/2024 e 31/12/2024
dates = pd.date_range(start="2024-01-01", end="2024-12-31", freq="h")
dates = np.random.choice(dates, n, replace=True)

In [5]:
# Simular sazonalidade: mais pedidos em novembro e dezembro, menos em janeiro e fevereiro
for i in range(n):
    mes = pd.Timestamp(dates[i]).month
    if mes in [11, 12]:  # Black Friday e Natal
        if np.random.rand() < 0.5:  # 50% de chance de criar um pedido extra
            dates[i] = np.random.choice(
                pd.date_range("2024-11-01", "2024-12-31", freq="h")
            )
    elif mes in [1, 2]:  # Período de baixa demanda
        if np.random.rand() < 0.3:  # 30% de chance de remover um pedido
            dates[i] = np.random.choice(
                pd.date_range("2024-01-01", "2024-02-28", freq="h")
            )

In [6]:
# Centros de distribuição
centros = [
    "São Paulo (SP)",
    "Recife (PE)",
    "Brasília (DF)",
    "Florianópolis (SC)",
    "Belém (PA)",
]
centros_cd = np.random.choice(centros, n)

In [7]:
# Percentual de ocupação do CD no momento do pedido (%)
percentual_ocupacao_CD = np.random.normal(loc=75, scale=15, size=n)
percentual_ocupacao_CD = np.clip(percentual_ocupacao_CD, 40, 100)

In [8]:
# Tempo de separação aumenta se a ocupação do CD for alta
tempo_separacao = (
    np.random.normal(loc=20, scale=5, size=n) + (percentual_ocupacao_CD - 75) * 0.2
)
tempo_separacao = np.clip(tempo_separacao, 5, 60)

In [9]:
# Tempo de embalagem varia conforme o tamanho do pedido
quantidade_itens = np.random.randint(1, 10, n)
tempo_embalagem = np.random.normal(loc=10, scale=3, size=n) + quantidade_itens * 0.5
tempo_embalagem = np.clip(tempo_embalagem, 2, 30)

In [10]:
# Tempo total de processamento
tempo_total = tempo_separacao + tempo_embalagem + np.random.randint(5, 20, n)

In [11]:
# Status do pedido: 15% de chance de atraso, mas maior quando o CD está superlotado
prob_atraso = np.where(
    percentual_ocupacao_CD > 85, 0.25, 0.15
)  # Maior atraso se ocupação > 85%
status_pedido = np.where(np.random.rand(n) < prob_atraso, "Atrasado", "Entregue")

In [12]:
# Tempo de atraso no transporte (apenas para pedidos atrasados)
atraso_transporte_min = np.where(
    status_pedido == "Atrasado", np.random.randint(10, 120, n), 0
)

In [13]:
# Erro na separação do pedido (1 = sim, 0 = não) - Mais comum em picos de demanda
erro_picking = np.random.choice([0, 1], n, p=[0.95, 0.05])

In [14]:
# Categorias de produto
categorias = ["Eletrodoméstico", "Vestuário", "Alimentos", "Eletrônicos", "Brinquedos"]
categoria_produto = np.random.choice(categorias, n)

In [15]:
# Peso total do pedido (kg) - Produtos maiores exigem mais tempo
peso_total_kg = np.random.uniform(0.5, 30.0, n)

In [16]:
# Criar DataFrame
df = pd.DataFrame(
    {
        "id_pedido": np.arange(1, n + 1),
        "data_hora_pedido": dates,
        "centro_distribuicao": centros_cd,
        "percentual_ocupacao_CD": np.round(percentual_ocupacao_CD, 2),
        "tempo_separacao_min": np.round(tempo_separacao, 2),
        "tempo_embalagem_min": np.round(tempo_embalagem, 2),
        "tempo_total_processamento_min": np.round(tempo_total, 2),
        "status_pedido": status_pedido,
        "atraso_transporte_min": atraso_transporte_min,
        "erro_picking": erro_picking,
        "categoria_produto": categoria_produto,
        "quantidade_itens": quantidade_itens,
        "peso_total_kg": np.round(peso_total_kg, 2),
    }
)

In [17]:
# Exibir as primeiras linhas
print(df.head())

   id_pedido    data_hora_pedido centro_distribuicao  percentual_ocupacao_CD  \
0          1 2024-01-25 01:00:00  Florianópolis (SC)                   61.23   
1          2 2024-02-12 23:00:00  Florianópolis (SC)                  100.00   
2          3 2024-10-26 05:00:00         Recife (PE)                   81.07   
3          4 2024-01-08 14:00:00      São Paulo (SP)                   67.09   
4          5 2024-02-06 03:00:00      São Paulo (SP)                   99.72   

   tempo_separacao_min  tempo_embalagem_min  tempo_total_processamento_min  \
0                19.91                10.99                          49.90   
1                22.99                19.46                          56.45   
2                20.64                16.20                          42.84   
3                14.88                10.02                          36.90   
4                21.39                17.99                          44.38   

  status_pedido  atraso_transporte_min  erro_picki

In [18]:
#  Remover valores duplicados
df = df.drop_duplicates()

In [19]:
# Verificar e tratar valores nulos
df = df.dropna()

In [20]:
# Converter tipos de dados
df["data_hora_pedido"] = pd.to_datetime(df["data_hora_pedido"])
df["centro_distribuicao"] = df["centro_distribuicao"].astype("category")
df["status_pedido"] = df["status_pedido"].astype("category")
df["categoria_produto"] = df["categoria_produto"].astype("category")

In [21]:
# Salvar dataset processado
df.to_csv(os.path.join(PATH_RESOURCES, "dados_logisticos_sinteticos.csv"), index=False)

In [22]:
# Exibir informações sobre o dataset processado
print("\nInformações do Dataset Processado:")
print(df.info())
print("\nEstatísticas Descritivas:")
print(df.describe())


Informações do Dataset Processado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id_pedido                      50000 non-null  int64         
 1   data_hora_pedido               50000 non-null  datetime64[ns]
 2   centro_distribuicao            50000 non-null  category      
 3   percentual_ocupacao_CD         50000 non-null  float64       
 4   tempo_separacao_min            50000 non-null  float64       
 5   tempo_embalagem_min            50000 non-null  float64       
 6   tempo_total_processamento_min  50000 non-null  float64       
 7   status_pedido                  50000 non-null  category      
 8   atraso_transporte_min          50000 non-null  int64         
 9   erro_picking                   50000 non-null  int64         
 10  categoria_produto              50000 non-null 